<a href="https://colab.research.google.com/github/jmccahill5/datman/blob/main/PS2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#---------------------------SETUP----------------------------------
#get useful libraries
import time, os, sys, re #basics
import zipfile, json, datetime, string   #string for annotating points in scatter
import numpy as np #basic math
from statistics import * #stats

import matplotlib.pyplot as plt #import pylab as plt #apparently discouraged now:
 #https://stackoverflow.com/questions/11469336/what-is-the-difference-between-pylab-and-pyplot
 #https://www.tutorialspoint.com/matplotlib/matplotlib_pylab_module.htm

import pandas as pd
import pandas_datareader as pdr
from pandas_datareader import wb
from pandas.io.formats.style import Styler
#s4 = Styler(df4, uuid_len=0, cell_ids=False)

from google.colab import files

#import webbrowser

import seaborn as sns

from google.colab import data_table
data_table.enable_dataframe_formatter() #this enables spreadsheet view upon calling dataframe (without() )


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


%matplotlib inline




In [ ]:
#1st dataset

In [49]:
!wget -q -O ex1.csv https://github.com/jmccahill5/datman/raw/main/demand.csv

In [50]:

!pwd

!ls

/content
ex1.csv  med.csv  sample_data


In [51]:
dat=pd.read_csv('ex1.csv')

In [52]:
dat.columns

Index(['DATE', 'CSUSHPISA', 'MORTGAGE30US', 'UMCSENT', 'INTDSRUSM193N',
       'MSPUS', 'GDP'],
      dtype='object')

In [ ]:
#What each variable means
#INTDSRUSM193N	Interest Rates, Discount Rate for United States (Billions of Dollars, Seasonally Adjusted Annual Rate)
#UMCSENT	University of Michigan: Consumer Sentiment
#GDP	Gross Domestic Product (Billions of Dollars, Seasonally Adjusted Annual Rate)
#MORTGAGE15US	30-Year Fixed Rate Mortgage Average in the United States (Percent, Not Seasonally Adjusted)
#CSUSHPISA	S&P/Case-Shiller U.S. National Home Price Index (Index Jan 2000=100, Seasonally Adjusted)
#MSPUS	Median Sales Price of Houses Sold for the United States (Not Seasonally Adjusted)

In [53]:
dat.head(10)

,DATE,CSUSHPISA,MORTGAGE30US,UMCSENT,INTDSRUSM193N,MSPUS,GDP
0,1/1/2003,129.321000,5.840769,79.966667,2.250000,186000,11174.129
1,1/4/2003,131.756000,5.506923,89.266667,2.166667,191800,11312.766
2,1/7/2003,135.013000,6.033846,89.300000,2.000000,191900,11566.669
3,1/10/2003,138.835667,5.919286,91.966667,2.000000,198800,11772.234
4,1/1/2004,143.298667,5.597500,98.000000,2.000000,212700,11923.447
5,1/4/2004,148.209000,6.106154,93.333333,2.083333,217600,12112.815
6,1/7/2004,152.716667,5.894286,95.600000,2.500000,213500,12305.307
7,1/10/2004,157.535333,5.736154,93.866667,3.000000,228800,12527.214
8,1/1/2005,163.481333,5.768462,94.066667,3.500000,232500,12767.286
9,1/4/2005,169.347333,5.708462,90.200000,4.000000,233700,12922.656


In [54]:
dat["CSUSHPISA"].mean()

180.658712495

In [ ]:
#2nd dataset

In [55]:
!wget -q -O ex1.csv https://github.com/jmccahill5/datman/raw/main/supply.csv

In [56]:

!pwd

!ls

/content
ex1.csv  med.csv  sample_data


In [57]:
dat2=pd.read_csv('ex1.csv')

In [58]:
dat2.columns

Index(['DATE', 'MSACSR', 'PERMIT', 'TLRESCONS', 'EVACANTUSQ176N'], dtype='object')

In [60]:
dat2.head(10)

,DATE,MSACSR,PERMIT,TLRESCONS,EVACANTUSQ176N
0,1/1/2003,4.2,1806.333333,421328.6667,14908
1,1/4/2003,3.833333333,1837.666667,429308.6667,15244
2,1/7/2003,3.633333333,1937.333333,458890,15614
3,1/10/2003,3.966666667,1972.333333,491437.3333,15654
4,1/1/2004,3.7,1994.666667,506856.3333,15895
5,1/4/2004,3.9,2080,526483.3333,16079
6,1/7/2004,4.3,2069.666667,552939,15656
7,1/10/2004,4.1,2086,561413.3333,15361
8,1/1/2005,4.266666667,2105,590790.3333,15748
9,1/4/2005,4.266666667,2137.666667,611957,16061


In [ ]:
## what each variable name stands for
#PERMIT	New Privately-Owned Housing Units Authorized in Permit-Issuing Places: Total Units (Thousands of Units, Seasonally Adjusted Annual Rate)
#MSACSR	Monthly Supply of New Houses in the United States (Seasonally Adjusted)
#TLRESCONS	Total Construction Spending: Residential in the United States (Millions of Dollars, Seasonally Adjusted Annual Rate)
#EVACANTUSQ176N	Housing Inventory Estimate: Vacant Housing Units in the United States (Thousands of Units, Not Seasonally Adjusted)
#CSUSHPISA	S&P/Case-Shiller U.S. National Home Price Index (Index Jan 2000=100, Seasonally Adjusted

In [ ]:
#merging the data into one set

In [59]:
mergedOuter = pd.merge(dat, dat2, how='outer', on=['DATE'],indicator=True) #outer=keep all
mergedOuter

,DATE,CSUSHPISA,MORTGAGE30US,UMCSENT,INTDSRUSM193N,MSPUS,GDP,MSACSR,PERMIT,TLRESCONS,EVACANTUSQ176N,_merge
0,1/1/2003,129.321000,5.840769,79.966667,2.250000,186000.0,11174.129,4.2,1806.333333,421328.6667,14908,both
1,1/4/2003,131.756000,5.506923,89.266667,2.166667,191800.0,11312.766,3.833333333,1837.666667,429308.6667,15244,both
2,1/7/2003,135.013000,6.033846,89.300000,2.000000,191900.0,11566.669,3.633333333,1937.333333,458890,15614,both
3,1/10/2003,138.835667,5.919286,91.966667,2.000000,198800.0,11772.234,3.966666667,1972.333333,491437.3333,15654,both
4,1/1/2004,143.298667,5.597500,98.000000,2.000000,212700.0,11923.447,3.7,1994.666667,506856.3333,15895,both
...,...,...,...,...,...,...,...,...,...,...,...,...
77,1/4/2022,303.422667,5.266154,57.866667,NaN,449300.0,25248.476,8.7,1734.666667,947300.3333,15286,both
78,1/7/2022,301.726333,5.623077,56.100000,NaN,468000.0,25723.941,9.566666667,1610.666667,910346,15306,both
79,1/10/2022,297.896667,6.664615,58.800000,NaN,479500.0,26137.992,9.2,1455.333333,870620.6667,14554,both
80,1/1/2023,NaN,6.372308,64.633333,NaN,436800.0,26465.865,8.033333333,1424.333333,842559.6667,15053,both
